In [16]:
import sys
sys.path.append("../../../tket2-rewriting/tket2_rewriting")
from circuit_generation.gadget import CXGadgetBenchmarkCircuit,SimpleLabelFriendlyCircuit
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
from three_qubit_squash.three_qubit_squash_gadget import Squash3Gadget
from pytket import Circuit, OpType
from pytket.passes import DecomposeBoxes, AutoRebase
from pytket import OpType
from tket.circuit import Tk2Circuit
from tket.optimiser import BadgerOptimiser

In [17]:
opt = BadgerOptimiser([Squash3Gadget()])

In [18]:

def boundary_ports_between_barriers(circ: Circuit):
    """
    Return (ports, (start_idx, end_idx), circ_used).
    If circ has <2 barriers, it builds a bracketed copy: [BARRIER] + circ + [BARRIER].
    """
    def _compute_ports(c, sidx, eidx):
        cmds = list(c.get_commands())
        mid = cmds[sidx+1:eidx]
        touched = set()
        for cmd in mid:
            for u in cmd.args:
                touched.add(u)
        return [q for q in c.qubits if q in touched]

    cmds = list(circ.get_commands())
    barrier_idxs = [i for i, cmd in enumerate(cmds) if cmd.op.type == OpType.Barrier]

    if len(barrier_idxs) >= 2:
        start_idx, end_idx = barrier_idxs[0], barrier_idxs[-1]
        return _compute_ports(circ, start_idx, end_idx), (start_idx, end_idx), circ

    # Auto-bracket: [BARRIER] + circ + [BARRIER]
    c2 = Circuit(circ.n_qubits)
    c2.add_barrier(c2.qubits)
    # qubit mapping
    c2.add_circuit(circ, c2.qubits)
    c2.add_barrier(c2.qubits)

    cmds2 = list(c2.get_commands())
    barrier_idxs2 = [i for i, cmd in enumerate(cmds2) if cmd.op.type == OpType.Barrier]
    start_idx, end_idx = barrier_idxs2[0], barrier_idxs2[-1]
    return _compute_ports(c2, start_idx, end_idx), (start_idx, end_idx), c2

In [19]:
orig = SimpleLabelFriendlyCircuit(6, 6)

cprep = orig.copy()
DecomposeBoxes().apply(cprep)
AutoRebase({OpType.CX, OpType.Rz, OpType.Rx}).apply(cprep)

False

In [20]:
before_ports, (sidx, eidx), circ_used = boundary_ports_between_barriers(cprep)

In [21]:
opt = BadgerOptimiser([Squash3Gadget()])
tk2 = Tk2Circuit(circ_used.copy())

In [22]:
_ = opt.optimise(tk2)

In [23]:
after = tk2.to_tket1()

In [24]:
after_ports, _, _ = boundary_ports_between_barriers(after)

In [25]:
print("Boundary ports before:", before_ports)
print("Boundary ports after :", after_ports)
print("Ports preserved?      ", before_ports == after_ports)

Boundary ports before: [q[0], q[1], q[2], q[3], q[4], q[5]]
Boundary ports after : [q[0], q[1], q[2], q[3], q[4], q[5]]
Ports preserved?       True
